# What causes a molecule to be toxic and how can we predict it?


The “Toxicology in the 21st Century” (Tox21) initiative created a public database measuring toxicity of compounds, which has been used in the 2014 Tox21 Data Challenge. This dataset contains qualitative toxicity measurements for 8k compounds on 12 different targets, including nuclear receptors and stress response pathways.

Random splitting is recommended for this dataset.

The raw data csv file contains columns below:

“smiles” - SMILES representation of the molecular structure

“NR-XXX” - Nuclear receptor signaling bioassays results

“SR-XXX” - Stress response bioassays results

please refer to https://tripod.nih.gov/tox21/challenge/data.jsp for details.

This problem is a classification problem: Toxic/Non-toxic. 

References:
1. https://moleculenet.org/ website and paper on the datasets
2. https://deepchem.readthedocs.io/en/latest/index.html manual
3. https://github.com/deepchem/deepchem/tree/master/examples/tutorials tutorials


In [ ]:
# lets load our libraries
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import Draw
import tensorflow as tf

import pandas as pd

from matplotlib import pyplot as plt

import numpy as np

from sklearn import metrics
from sklearn.metrics import f1_score

print("TensorFlow version: " + tf.__version__)
print("DeepChem version: " + dc.__version__)

# Loading the dataset example

In [ ]:
# this line loads the  dataset - we are using the extended connectivity fingerprints here, 
# there are several featurizers for you to try
# the splitter splits the dataset for you, refer to the MoleculeNet paper to see which one you need
tasks, datasets, transformers = dc.molnet.load_tox21(
    featurizer='ECFP', 
    splitter='random')
# the datasets object is already split into the train, validation and test dataset 
train_dataset, valid_dataset, test_dataset = datasets
## N.B. Some molecules may not featurize and you'll get a warning this is OK

In [ ]:
# Dataset contains the input fingerprints as X, the output is stored in y, and the IDS are the smiles strings
print(test_dataset)

# Using RDKit

RDkit is a chemistry package that allows you to create chemical features or do very simple computational chemistry.

In [ ]:
# and this draws a nice image of the first 12 molecules of the test_dataset
SMILES_strings = test_dataset.ids[:12]
ms = [Chem.MolFromSmiles(x) for x in SMILES_strings] 
Draw.MolsToGridImage(ms)

RDKit does not give us just a picture of the molecule, it creates a molecule object:

In [ ]:
molecule_number = 1
formula = SMILES_strings[molecule_number]
a_molecule = Chem.MolFromSmiles(formula)
a_molecule

The function `dir()` gives a list of the possible things that object can do. (Ignore those that start with ``__`` these are internal functions that you want to leave alone). For example, you could run `a_molecule.GetNumAtoms` to get the number of atoms. More details of what you can calculated with RDKit is here:
https://rdkit.org/docs/index.html

In [ ]:
dir(a_molecule)

In [ ]:
# Help brings up the manual for the object
help(a_molecule)

In [ ]:
a_molecule.GetNumAtoms()

# More about the dataset
The input, `X`, is a fingerprint, a list of `1`s and `0`s for wether a feature is present or not (a feature like a functional group). There are other types of input that you get from other featurizers that are needed for different ML models.

The output `y` is the is molecule's toxicity against various targets.

In [ ]:
test_dataset.X[molecule_number]

The `1.`s below indicate that this model failed a toxicity test! I.e. it was toxic against that receptor.

In [ ]:
test_dataset.y[molecule_number]

In [ ]:
# the molecule again.
a_molecule

These are the tasks:

In [ ]:
test_dataset.tasks

#### How many toxic molecules do we have?

 You can investigate the data, for example, below is the count of toxic drug candiates 

In [ ]:
# you will probably want to do this for all the datasets
# but lets just look at the test dataset here
# this is the number of toxic molecules for each task
sum(test_dataset.y)

In [ ]:
#This is how many molecules there are in this dataset
len(test_dataset)

In [ ]:
# Here we plot the percentages of toxic molecules per task
ax=plt.bar(train_dataset.tasks[:7],100*sum(train_dataset.y[:,:7])/len(train_dataset))
plt.bar(train_dataset.tasks[7:],100*sum(train_dataset.y[:,7:])/len(train_dataset))
plt.xticks(rotation=-45, ha='left')
plt.ylabel("Toxic molecules %")
#plt.xlabel("Tzxcasks")
plt.title("Test Dataset")

### THIS IS A SMALL NUMBER OF TOXIC MOLECULES!

This is a very unbalanced dataset, you only have around 5-10% toxic molecules and the rest are non-toxic, this is an unbalanced dataset.

In [ ]:
# the nuclear reactor tasks
test_dataset.tasks[:7]

In [ ]:
# the stress receptor tasks
test_dataset.tasks[7:]

# Machine learning example
(see also the 4th and 5th notebooks in the Machine Learning for Chemist's course).
The code below uses one of `deepchem`'s models. There are also relevant models in the `sklearn` module. There are more options that be tuned. The process of changing these options is called *hyperparameter optimisation*.

Here we train a classifier that will work on more than one task at a time. Classifiers put molecules into two classes: toxic or not toxic and multiclass classifiers do it on a per assay basis.

Here we train a MultitaskClassifier on the NR datasets

In [ ]:
# this loads in a general purpose regression model
model = dc.models.MultitaskClassifier(
    n_tasks = len(test_dataset.tasks), # size of y, we have one output task here: finding toxicity
    n_features = len(test_dataset.X[0]), # number of input features, i.e. the length of the ECFPs
    layer_size = [1000],
    weight_init_stddevs = 0.02,
    bias_init_consts = 1.0,
    weight_decay_penalty = 0.001,
    weight_decay_penalty_type = 'l2',
    dropouts = 0.2,
    n_classes = 2,
    residual = False
)

############################################
# Now we fit the training dataset!         #
############################################
model.fit(train_dataset, nb_epoch=10)

This tests the trained model. Classifiers are usually tested by looking at the Area Under the Curve of the Receiver Operator Curve or **ROC AUC**, these are the numbers in deepchem that you will compare to. ROC AUC goes from 0 to 1, 0.5 means it's as good as random guessing, 1 is perfect, and anything less than 0.5 means it's worse than guessing.

In [ ]:
# this line tells deepchem what metric to use to score the datasets
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)

# model.evaluate() tests the model. 
# we have to give it the data to use, the metric (or set of metrics) and the transformer used
print('Over all tasks:')
print("Training set score:", model.evaluate(train_dataset, [metric], transformers))
print("Valid set score:", model.evaluate(valid_dataset, [metric], transformers))
print("Test set score:", model.evaluate(test_dataset, [metric], transformers))
ground_truth = test_dataset.y
predictions = model.predict(test_dataset)
metric = dc.metrics.roc_auc_score
print('Per task:')
print('Task\t\tROC AUC')
for i in range(len(test_dataset.tasks)):
    score = metric(dc.metrics.to_one_hot(ground_truth[:,i]), predictions[:,i])
    print('{:}\t\t{:.3}'.format(test_dataset.tasks[i],score))

In [ ]:
task_names=tasks

In [ ]:
def cutoff(x):
    if x>0.55:
        return 1
    else:
        return 0

In [ ]:
#task_names = ['SR-ARE','SR-ATAD5','SR-HSE','SR-MMP','SR-p53']
for task_number in range (5):
    metrics.RocCurveDisplay.from_predictions(ground_truth[:,task_number], predictions[:,task_number][:,1])
    plt.plot([0,1], [0,1])
    plt.axis('square')
    plt.title(test_dataset.tasks[task_number])
    print('f1_score of {} is'.format(task_names[task_number]), f1_score(ground_truth[:,task_number], [cutoff(x) for x in predictions[:,task_number][:,1]], average='binary'))
    print('No. of predicted toxic molecules of {} is'.format(task_names[task_number]), sum([cutoff(x) for x in predictions[:,task_number][:,1]]))
    print('Ground truth of {} is'.format(task_names[task_number]), sum(test_dataset.y)[task_number])
    print('')

Here is a ROC AUC for one of the tasks. It looks OK. 

In [ ]:
task_number = 1
metrics.RocCurveDisplay.from_predictions(ground_truth[:,task_number], predictions[:,task_number][:,1])
plt.plot([0,1], [0,1])
plt.axis('square')
plt.title(test_dataset.tasks[task_number])

But, in this field you can score the accuracy of your model in several ways, the metric you use for classificiation where you have unbalanced datasets is the F1 score (there are also other valid choices). And the f1 score for this task is terrible! (F1 goes between 0 and 1)

In [ ]:
f1_score(ground_truth[:,task_number], [round(x) for x in predictions[:,task_number][:,1]], average='binary')

This code counts how many molecules were predicted to be toxic:

In [ ]:
sum([round(x) for x in predictions[:,task_number][:,1]])

This model has solved the problem by just saying everything is non-toxic, this is not what we want!

## Training with early stopping

Rather than training for a set number of epochs, it is better to use early stopping. This will train until the point where the valdiation results start to get worse. This avoids overtraining.

In [ ]:
# this is how many epochs wait to see if the training gets better
patience = 15
# these are some nice metrics
metric1 = dc.metrics.Metric(dc.metrics.balanced_accuracy_score)
metric2 = dc.metrics.Metric(dc.metrics.prc_auc_score)
metric3 = dc.metrics.Metric(dc.metrics.roc_auc_score)
metric4 = dc.metrics.Metric(dc.metrics.f1_score)
#metric3 = dc.metrics.Metric(dc.metrics.mae_score)
metrics = [metric1, metric2, metric3, metric4]
selected_metric = 2 #which metric to use for callback 
        #i.e. which to train with
metric_selector=2
metric_labels = ['balanced_accuracy_score',#0
                 'prc_auc_score',#1
                 'roc_auc_score',#2
                 'f1_score']#3

In [ ]:
# this loads in a general purpose regression model
model = dc.models.MultitaskClassifier(
    n_tasks = len(test_dataset.tasks), # size of y, we have one output task here: finding toxicity
    n_features = len(test_dataset.X[0]), # number of input features, i.e. the length of the ECFPs
    layer_size = [1000,500],
    weight_init_stddevs = 0.02,
    bias_init_consts = 1.0,
    weight_decay_penalty = 0.001,
    weight_decay_penalty_type = 'l2',
    dropouts = 0.2,
    n_classes = 2,
    residual = False
)

# this sets up a callback on the validation
callback = dc.models.ValidationCallback(
            valid_dataset,
            patience,
            metrics[metric_selector])
# fit da model
model.fit(train_dataset, nb_epoch=2, callbacks=callback)



## This is a nice function to create the datasets

In [ ]:
def get_them_metrics(
        model,
        datasets,
        metrics,
        metric_labels,
        transformers=[],
):
    """calculates metrics for a run
    model: trained model
    # datasets: tuple of datasets
    # metrics: list of metric objects
    # metric labels: sensible labels"""
    ugh = []
    for dataset in datasets:
        if transformers == []:
            egg = model.evaluate(
                dataset,
                metrics)
        else:
            egg = model.evaluate(
                dataset,
                metrics,
                transformers=transformers)
        for metric_label in metric_labels:
            if metric_label == 'rmse':
                ugh.append(np.sqrt(egg['mean_squared_error']))
            else:
                ugh.append(egg[metric_label])
    return ugh

In [ ]:
# little function to calc metrics on this data
out=get_them_metrics(
            model,
            datasets,
            metrics,
            metric_labels,
            transformers)
# makes a nice dataframe
pd_out = pd.DataFrame([out], columns=['tr_mse', 'tr_r2', 'tr_mae', 'tr_rmse',
                                        'val_mse', 'val_r2', 'val_mae', 'val_rmse',
                                        'te_mse', 'te_r2', 'te_mae', 'te_rmse'])
pd_out

You can save this as a .csv file using `pd_out.to_csv('filename.csv')`

# Training models

This approach (ECFP + multiclass classifier) isn't very good, it's gotten nice AUC ROC curves but this is misleading. You will have to have a look at how to train with unbalanced classes. 

See for example:
https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-imbalanced-classification/
https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/
https://machinelearningmastery.com/precision-recall-and-f-measure-for-imbalanced-classification/
and the balancing and unbalancing transformers
https://deepchem.readthedocs.io/en/latest/api_reference/transformers.html?highlight=dc.trans.BalancingTransformer#deepchem.trans.BalancingTransformer

Now go and have a play